In [23]:
### basic libraries
import os
import itertools
from datetime import datetime
from json import dumps
import pandas as pd 
import numpy as np

### carto libraries
import shapefile ### not necessary ?
import geopandas as gp
#from shapely.geometry import Polygon
from shapely.geometry import shape, Point

### for plotting in Jupyter
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns
plt.rcParams["figure.figsize"] = (10.0, 10.0)
#plt.style.use("bmh")
plt.style.use("ggplot")

#from app.scripts.topojson import topojson

from pyproj import Proj, transform ### not necessary ?

crs_lambert93 = 2154
crs_WSG84     = 4326
inCRS  = 'epsg:%s' %(crs_lambert93)
outCRS = 'epsg:%s' %(crs_WSG84)
inProj  = Proj(init=inCRS)  # proj in  : Lambert 93
outProj = Proj(init=outCRS) # proj out : WSG 84



In [24]:
### basic folders addresses and names

cwd = os.getcwd()

data_folder      = "app/static/data"

stats_folder     = "stats"
_web             = "_web" 
stats_web_folder = stats_folder + _web

stats_path     = os.path.join(cwd, data_folder, stats_folder)
stats_web_path = os.path.join(cwd, data_folder, stats_web_folder)


carto_folder     = "carto"
_web             = "_web"
carto_web_folder = carto_folder + _web

carto_path     = os.path.join(cwd, data_folder, carto_folder)
carto_web_path = os.path.join(cwd, data_folder, carto_web_folder)



print "-- cwd :"            , cwd
print "-- stats_web_path : ", stats_web_path
print "-- carto_web_path : ", carto_web_path

-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- stats_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web
-- carto_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto_web


In [65]:
### read villes_france.csv from : data/stats_web

#_sep_csv         = ";"
csv_encoding_web = "utf-8"

villes_filename = "villes_france.csv"
villes_web_filepath = os.path.join(stats_web_path, villes_filename)
print villes_web_filepath

df_villes_web = pd.read_csv( villes_web_filepath, sep=",", encoding=csv_encoding_web )

df_villes_web.head(3)

/Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web/villes_france.csv


,code postal,insee,article,ville,ARTICLE,VILLE,libelle,region,nom region,dep,nom dep,longitude,latitude,codex,metaphone
0,1500,01004,NaN,Ambérieu-en-Bugey,NaN,AMBERIEU-EN-BUGEY,AMBERIEU EN BUGEY,82,RHONE-ALPES,01,Ain,45.979851,5.336890,A516,AMPRNPJ
1,1330,01005,NaN,Ambérieux-en-Dombes,NaN,AMBERIEUX-EN-DOMBES,AMBERIEUX EN DOMBES,82,RHONE-ALPES,01,Ain,45.998057,4.902498,A516,AMPRKSNTMPS
2,1300,01006,NaN,Ambléon,NaN,AMBLEON,AMBLEON,82,RHONE-ALPES,01,Ain,45.749870,5.601326,A514,AMPLN


In [66]:
### create point geometries

geometry     = [ Point(xy) for xy in zip(df_villes_web.latitude, df_villes_web.longitude) ]

gdf_villes = gp.GeoDataFrame(df_villes_web, crs=crs_WSG84, geometry=geometry)

gdf_villes.tail(3)

,code postal,insee,article,ville,ARTICLE,VILLE,libelle,region,nom region,dep,nom dep,longitude,latitude,codex,metaphone,geometry
37007,75000,75056,NaN,Paris,NaN,PARIS,PARIS,11,ILE-DE-FRANCE,75,Paris,48.856918,2.34121,P620,PRS,POINT (2.34121 48.856918)
37008,13540,13921,NaN,PUYRICARD,NaN,PUYRICARD,PUYRICARD,93,PROVENCE-ALPES-COTE D'AZUR,13,Bouches-du-Rhône,43.582729,5.42192,P626,PRKRT,POINT (5.42192 43.582729)
37009,59160,59355,NaN,LOMME,NaN,LOMME,LOMME,31,NORD-PAS-DE-CALAIS,59,Nord,50.643559,2.98782,L500,LM,POINT (2.98782 50.643559)


In [67]:
#import fiona
#fiona.crs.from_epsg(4326)

In [68]:
### set projection CRS WSG84 

gdf_villes.crs = {
                    'init' : 'epsg:%s' %(crs_WSG84),
                    'no_defs': True
                   }

print gdf_villes.crs


{'init': 'epsg:4326', 'no_defs': True}


In [85]:
# work on copy
gdf_villes_json = gdf_villes.copy()

### set indexes :

gdf_villes_json.set_index([
                        #"code postal", 
                        "insee", 
                       ], inplace=True, drop=False) 
#gdf_stations.set_index("CD_STATION", inplace=True) 
gdf_villes_json.sort_index(inplace=True) 

print "-- gdf_villes_json.shape : ", gdf_villes_json.shape

-- gdf_villes_json.shape :  (37010, 16)


In [86]:
gdf_villes_json.head(7)

,code postal,insee,article,ville,ARTICLE,VILLE,libelle,region,nom region,dep,nom dep,longitude,latitude,codex,metaphone,geometry
insee,,,,,,,,,,,,,,,,
01001,1400,01001,L',Abergement-Clémenciat,L',ABERGEMENT-CLEMENCIAT,L ABERGEMENT CLEMENCIAT,82,RHONE-ALPES,01,Ain,46.151468,4.920059,A162,APRJMNTKLMNSTL,POINT (4.920058999999999 46.151468)
01002,1640,01002,L',Abergement-de-Varey,L',ABERGEMENT-DE-VAREY,L ABERGEMENT DE VAREY,82,RHONE-ALPES,01,Ain,46.005930,5.425281,A162,APRJMNTTFRL,POINT (5.425281 46.00593)
01004,1500,01004,NaN,Ambérieu-en-Bugey,NaN,AMBERIEU-EN-BUGEY,AMBERIEU EN BUGEY,82,RHONE-ALPES,01,Ain,45.979851,5.336890,A516,AMPRNPJ,POINT (5.33689 45.979851)
01005,1330,01005,NaN,Ambérieux-en-Dombes,NaN,AMBERIEUX-EN-DOMBES,AMBERIEUX EN DOMBES,82,RHONE-ALPES,01,Ain,45.998057,4.902498,A516,AMPRKSNTMPS,POINT (4.902498 45.998057)
01006,1300,01006,NaN,Ambléon,NaN,AMBLEON,AMBLEON,82,RHONE-ALPES,01,Ain,45.749870,5.601326,A514,AMPLN,POINT (5.601326 45.74987)
01007,1500,01007,NaN,Ambronay,NaN,AMBRONAY,AMBRONAY,82,RHONE-ALPES,01,Ain,46.006480,5.359990,A516,AMPRN,POINT (5.35999 46.00648)
01008,1500,01008,NaN,Ambutrix,NaN,AMBUTRIX,AMBUTRIX,82,RHONE-ALPES,01,Ain,45.939594,5.338195,A513,AMPTRKS,POINT (5.338195 45.939594)


In [87]:
print list(gdf_villes_json.columns)

[u'code postal', u'insee', u'article', u'ville', u'ARTICLE', u'VILLE', u'libelle', u'region', u'nom region', u'dep', u'nom dep', u'longitude', u'latitude', u'codex', u'metaphone', 'geometry']


In [88]:
### drop useless columns for geographic purposes

cols_to_drop = [
    u'article', 
    u'ARTICLE', 
    u'metaphone', 
    u'codex', 
    u'libelle', 
    u'ville', 
    u'longitude', 
    u'latitude', 
    u"region",
    u'nom dep',
    u'nom region',
    u'code postal',
    u'dep'
    ]
gdf_villes_json.drop(cols_to_drop, axis=1, inplace=True)


In [89]:
gdf_villes_json = gdf_villes_json.dropna( axis=0, how="all") # on empty rows


In [90]:
gdf_villes_json.shape

(37010, 3)

In [91]:
gdf_villes_json.head()

,insee,VILLE,geometry
insee,,,
01001,01001,ABERGEMENT-CLEMENCIAT,POINT (4.920058999999999 46.151468)
01002,01002,ABERGEMENT-DE-VAREY,POINT (5.425281 46.00593)
01004,01004,AMBERIEU-EN-BUGEY,POINT (5.33689 45.979851)
01005,01005,AMBERIEUX-EN-DOMBES,POINT (4.902498 45.998057)
01006,01006,AMBLEON,POINT (5.601326 45.74987)


In [92]:
### save geojson for villes


### filepath for villes_web_carto.geojson
out_filename = "villes_web_carto.geojson"
out_filepath = os.path.join(carto_web_path, out_filename )


### overule Fiona overwrite bug
try: 
    os.remove(out_filepath)
except OSError:
    pass


#fiona.supported_drivers
gdf_villes_json.to_file( out_filepath, driver="GeoJSON" )

